<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/Chap_7_transformers_bert_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## হাতেকলমে ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং: শুরুর ধারণা

[৭ম অধ্যায়](https://aiwithr.github.io/nlpbook/)

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "test" # username from the json file
os.environ['KAGGLE_KEY'] = "test"

In [ ]:
!kaggle datasets files furcifer/bangla-newspaper-dataset

name                  size  creationDate         
--------------------  ----  -------------------  
data_v2/data_v2.json   2GB  2020-10-21 06:09:48  
data/data.json         4GB  2020-10-21 06:09:48  


In [ ]:
!kaggle datasets download furcifer/bangla-newspaper-dataset -f data_v2/data_v2.json

100% 444M/444M [00:04<00:00, 120MB/s]



In [ ]:
!unzip data_v2.json.zip

Archive:  data_v2.json.zip
  inflating: data_v2.json            


In [ ]:
import json
with open('data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [ ]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [ ]:
categories = list(set(labels))

In [ ]:
categories

['life-style',
 'economy',
 'sports',
 'entertainment',
 'bangladesh',
 'education',
 'international',
 'technology',
 'opinion']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  sentences, labels, test_size=0.2, random_state=42)

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 140kB/s 
     |████████████████████████████████| 983kB 55.3MB/s 
     |████████████████████████████████| 266kB 55.2MB/s 
     |████████████████████████████████| 1.3MB 48.3MB/s 
     |████████████████████████████████| 1.1MB 47.2MB/s 
     |████████████████████████████████| 471kB 52.4MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
     |████████████████████████████████| 2.9MB 50.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.2-cp36-none-any.whl size=25276306 sha256=de1f7f0b4f1d4b74c8fee6da617ceef8c2812beee67e749bb00ea2189e996a47
  Stored in directory: /root/.cache/pip/wheels/fe/56/00/25444c352cc843e5c5daea0e9517a192878ae22c2c6b5f4573
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=ac1b0cab1773f739a06a112d03472243d7434505a32f91023147ab8cf52e9afc
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

In [ ]:
import ktrain
from ktrain import text

In [ ]:
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [ ]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 268
	95percentile : 669
	99percentile : 1079


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 269
	95percentile : 669
	99percentile : 1075


In [ ]:
model = t.get_classifier()

In [ ]:
# ট্রেনিং করে নিচ্ছি
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [16]:
%matplotlib inline
learner.lr_find(show_plot=True, max_epochs=1)

In [17]:
# preproc = t
# save model and Preprocessor instance after partially training
# ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')

In [18]:
# reload Predictor and extract model
# model = ktrain.load_predictor('/tmp/my_predictor').model

In [ ]:
learner.fit_onecycle(8e-5, 1)

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [20]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [21]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt')
# save model using transformers API after partially training
learner.model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor')


In [23]:
!ls -alh '/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt'

total 628M
-rw------- 1 root root  902 Dec 13 09:45 config.json
-rw------- 1 root root 628M Dec 13 09:45 tf_model.h5
-rw------- 1 root root 3.0K Dec 13 09:45 tf_model.preproc


In [25]:
learner.model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  164396544 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  6921      
Total params: 164,403,465
Trainable params: 164,403,465
Non-trainable params: 0
_________________________________________________________________


In [37]:
predictor.predict('প্রথম আলো সম্পাদকের আইনজীবী মোস্তাফিজুর রহমান খান সাংবাদিকদের জানান, গেল ৬ ডিসেম্বর প্রথম আলো সম্পাদকের বিরুদ্ধে মামলাটি বাতিল চেয়ে আবেদন করা হয়েছিল। এতে যুক্তি দেখানো হয়, মামলার তদন্ত প্রতিবেদনে মতিউর রহমানের সম্পৃক্ততার অভিযোগ ছিল না।')

'bangladesh'

In [38]:
predictor.explain('প্রথম আলো সম্পাদকের আইনজীবী মোস্তাফিজুর রহমান খান সাংবাদিকদের জানান, গেল ৬ ডিসেম্বর প্রথম আলো সম্পাদকের বিরুদ্ধে মামলাটি বাতিল চেয়ে আবেদন করা হয়েছিল। এতে যুক্তি দেখানো হয়, মামলার তদন্ত প্রতিবেদনে মতিউর রহমানের সম্পৃক্ততার অভিযোগ ছিল না।')

In [29]:
predictor.predict('বাংলাদেশ টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

'entertainment'

In [30]:
predictor.predict_proba('বাংলাদেশে টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

array([0.03908593, 0.006929  , 0.02715655, 0.6298372 , 0.01814232,
       0.05873922, 0.16282858, 0.04816281, 0.00911832], dtype=float32)

In [31]:
predictor.predict('বাংলাদেশে ফুটবল খুব বেশি খেলা হয়, ক্রিকেট কম নয়।')

'sports'

In [24]:
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-sw7oax9c
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-sw7oax9c
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106831 sha256=062164a107dbbb310f217ef058b415dbb52660a2e53d58b2e75e06b4614aadbb
  Stored in directory: /tmp/pip-ephem-wheel-cache-uwcqips_/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [ ]:
predictor.explain('শিক্ষা মন্ত্রণালয়ে এমন প্রস্তাব দিয়েছে মাউশি। মাউশির কর্মকর্তারা বলছেন, মন্ত্রণালয় অনুমোদন দিলেই সবাইকে সময়সহ অন্যান্য আনুষঙ্গিক বিষয় জানিয়ে দেওয়া হবে। তখন বেসরকারি বিদ্যালয়গুলোও কীভাবে নিজস্ব ব্যবস্থায় আবেদনপত্র বিতরণ ও লটারির কাজটি করবে, তা-ও জানিয়ে দেওয়া হবে। একই সঙ্গে ঢাকার বাইরের সরকারি বিদ্যালয়গুলোর বিষয়েও দিকনির্দেশনা দেওয়া হবে।')

In [ ]:
# টেন্সরফ্লোতে মডেল সেভ করা
# model.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(model, t).save('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [ ]:
!ls -alh '/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt'

In [ ]:
# reload the model using transformers directly
# from transformers import *
# model = TFAutoModelForSequenceClassification.from_pretrained('/tmp/my_model')
# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
reloaded_predictor = ktrain.load_predictor('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [ ]:
reloaded_predictor.predict_proba('বঙ্গবন্ধু টি–টোয়েন্টি কাপে কাল জেমকন খুলনার হয়ে খেলার ম্যাচে ৪ ওভারে ২৮ রানে ১ উইকেট নেন মাশরাফি বিন মুর্তজা।')

In [ ]:
reloaded_predictor.get_classes()